In [2]:
import pandas as pd  

if __name__ == "__main__":  
    now_phase = 6  
    train_path = r'underexpose_train'
    test_path = r'underexpose_test'  
    recom_item = [] 
    
    train_item_df = pd.read_csv(train_path+r'\underexpose_item_feat.csv',header=None)
    train_item_df.columns = ['item_id'] + ['txt_vec'+str(i) for i in range(128)] + ['img_vec'+str(i) for i in range(128)]
    train_item_df['txt_vec0'] = train_item_df['txt_vec0'].apply(lambda x:float(x[1:]))
    train_item_df['txt_vec127'] = train_item_df['txt_vec127'].apply(lambda x:float(x[:-1]))
    train_item_df['img_vec0'] = train_item_df['img_vec0'].apply(lambda x:float(x[1:]))
    train_item_df['img_vec127'] = train_item_df['img_vec127'].apply(lambda x:float(x[:-1]))

    whole_click = pd.DataFrame()  
    for c in range(now_phase + 1):  
        print('phase:', c)  
        click_train = pd.read_csv(train_path + r'\underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        click_test = pd.read_csv(test_path + r'\underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
  
        all_click = click_train.append(click_test)  
        whole_click = whole_click.append(all_click)  
        #all_click = all_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
        all_click = all_click.sort_values('time')


    tmp = whole_click.sort_values('time')
    tmp['next_item'] = tmp.groupby(['user_id'])['item_id'].transform(lambda x:x.shift(-1))
    tmp['pre_item'] = tmp.groupby(['user_id'])['item_id'].transform(lambda x:x.shift(1))
    next_item = tmp.groupby(['item_id','next_item'])['time'].agg({'count'}).reset_index().sort_values('count', ascending=False).astype(int)
    pre_item = tmp.groupby(['item_id','pre_item'])['time'].agg({'count'}).reset_index().sort_values('count', ascending=False).astype(int)
    next_item=next_item[next_item['next_item'].isin(train_item_df['item_id'])]
    pre_item=pre_item[pre_item['pre_item'].isin(train_item_df['item_id'])]

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5
phase: 6


In [3]:
'''
txt corr
img corr
'''
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
  
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    #user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))
    user_time_ = df.groupby(user_col)['time'].agg(list).reset_index() # 引入时间因素
    user_time_dict = dict(zip(user_time_[user_col], user_time_['time']))
    
    item_user_ = df.groupby(item_col)[user_col].agg(set).reset_index()  
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col])) 
    
    sim_item = {}  
    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):

        #items2=items.copy()
        nextitem=[]
        for i in items:
            temp_nextitem=next_item[next_item['item_id']==i]
            tempcount2=temp_nextitem[temp_nextitem['count']>=10]['next_item']#[:50]
            for j in tempcount2:
                if j not in items:
                    nextitem.append(j)
        nextitem=list(set(nextitem))
        #items=items2
        
        temp=train_item_df[train_item_df['item_id'].isin(items)]
        texttemp=temp.iloc[:,:129]
        texttemp.index=texttemp['item_id']
        texttemp=texttemp.drop(['item_id'],axis=1)
        texttempcorr=pd.DataFrame(texttemp.values.T, index=texttemp.columns, columns=texttemp.index).corr()
        
        imgtemp=temp[['item_id'] + ['img_vec'+str(i) for i in range(128)]]
        imgtemp.index=imgtemp['item_id']
        imgtemp=imgtemp.drop(['item_id'],axis=1)
        imgtempcorr=pd.DataFrame(imgtemp.values.T, index=imgtemp.columns, columns=imgtemp.index).corr()

        for loc1, item in enumerate(items):  
            item_cnt[item] += 1  
            sim_item.setdefault(item, {})  

            for loc2, relate_item in enumerate(items):  

                if item == relate_item:  
                    continue
                t1 = user_time_dict[user][loc1] # 点击时间提取
                t2 = user_time_dict[user][loc2]
                sim_item[item].setdefault(relate_item, 0)  
                if not use_iif:
                    sim_item[i][relate_item] += 1  
                #else:
                if (relate_item in texttempcorr.columns) & (item in texttempcorr.columns):

                    if loc1-loc2>0:
                        sim_item[item][relate_item] +=10*texttempcorr[item][relate_item]*imgtempcorr[item][relate_item] * 1 * 0.7 * (0.8**(loc1-loc2)) * (1 - (t1 - t2) * 5000) / math.log(1 + len(items))# 逆向

                    else:
                        sim_item[item][relate_item] +=10*texttempcorr[item][relate_item]*imgtempcorr[item][relate_item] * 1 * 1.0 * (0.8**(loc2-loc1)) * (1 - (t2 - t1) * 5000) / math.log(1 + len(items))# 正向

                else:
                    sim_item[item][relate_item] += 1 / math.log(1 + len(items))
 
        for loc3,item in enumerate(nextitem):  
            item_cnt[item] += 1  
            sim_item.setdefault(item, {})  

            for relate_item in items:  

                if item == relate_item:  
                    continue
                sim_item[item].setdefault(relate_item, 0)  
                if not use_iif:
                    sim_item[item][relate_item] += 1  

                if (relate_item in texttempcorr.columns) & (item in texttempcorr.columns):
                    sim_item[item][relate_item] +=  (8/math.sqrt(loc3))*texttempcorr[item][relate_item]*imgtempcorr[item][relate_item] / math.log(1 + len(items) + len(nextitem))
                else:
                    sim_item[item][relate_item] += 1 / math.log(1 + len(items) + len(nextitem))

    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij/(math.log(1+item_cnt[i])*math.log(1+item_cnt[j]))  #((item_cnt[i]*item_cnt[j])**0.2)
  
    return sim_item_corr, user_item_dict  


def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):

    rank = {}  
    interacted_items = user_item_dict[user_id] 
    interacted_items = interacted_items[::-1]
    for loc, i in enumerate(interacted_items):  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * (0.7**loc)

    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]
  
# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]

    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()
    return df  
  
  
if __name__ == "__main__":  
    now_phase = 6  
    train_path = r'underexpose_train'
    test_path = r'underexpose_test'  
    recom_item = [] 
    
    train_item_df = pd.read_csv(train_path+r'\underexpose_item_feat.csv',header=None)
    train_item_df.columns = ['item_id'] + ['txt_vec'+str(i) for i in range(128)] + ['img_vec'+str(i) for i in range(128)]
    train_item_df['txt_vec0'] = train_item_df['txt_vec0'].apply(lambda x:float(x[1:]))
    train_item_df['txt_vec127'] = train_item_df['txt_vec127'].apply(lambda x:float(x[:-1]))
    train_item_df['img_vec0'] = train_item_df['img_vec0'].apply(lambda x:float(x[1:]))
    train_item_df['img_vec127'] = train_item_df['img_vec127'].apply(lambda x:float(x[:-1]))

    whole_click = pd.DataFrame()  
    for c in range(now_phase + 1):  
        print('phase:', c)  
        click_train = pd.read_csv(train_path + r'\underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        click_test = pd.read_csv(test_path + r'\underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
  
        all_click = click_train.append(click_test)  
        whole_click = whole_click.append(all_click)  
        #all_click = all_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
        all_click = all_click.sort_values('time')
        item_sim_list, user_item = get_sim_item(all_click, 'user_id', 'item_id', use_iif=True)  
  
        for i in tqdm(click_test['user_id'].unique()):  
            rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
            for j in rank_item:  
                recom_item.append([i, j[0], j[1]])  

    # find most popular items  
    top50_click = whole_click['item_id'].value_counts().index[:50].values
    #top50_click = whole_click[whole_click['item_id'].isin(train_item_df['item_id'])]['item_id'].value_counts().index[50:100].values
    top50_click = ','.join([str(i) for i in top50_click])  
  
    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    result.to_csv(r'副本.csv', index=False, header=None)

phase: 0


100%|█████████████████████████████████████████████████████████████████████████████| 1663/1663 [00:12<00:00, 131.55it/s]


phase: 1


100%|█████████████████████████████████████████████████████████████████████████████| 1726/1726 [00:14<00:00, 119.00it/s]


phase: 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1708/1708 [00:22<00:00, 74.80it/s]


phase: 5


100%|██████████████████████████████████████████████████████████████████████████████| 1798/1798 [00:27<00:00, 66.34it/s]


phase: 6


100%|██████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:23<00:00, 77.50it/s]
